# Usage of Params

`Params` aims to be data transfer class. All flags, intermediate tensors, results should be transfered by instance of `Params`.

```python
param = Param(flags, tensors, results)
```

In [1]:
from pyscf.dh.util import Params, HybridDict

## Guide of using `Params`

1. `flags` contains running configurations. Usual dictionary.
   - Should be read-only for most situations, but no enforcement on this.
     If API user need to pass additional flags in functions,
     use `with param.with_flags_add({"add_flag": add_val})`.
   - Should contain simple types such as booleans, integers, enums, or tuples of those types.
     `flags` is at least serializable.
2. `tensors` contains intermediate matrices/tensors. `HybridDict` instance.
   - Should contain `h5py` instance, `np.ndarray` instance.
     Other types are strongly not recommended.
   - Scalar values are recommanded to be stored in `results` instead of `tensors`; user could also transfer scalar
     value to `np.ndarray` instance.
3. `results` contains outputs.
   - `results` should be serializable.

## Functional programming

For most computing extensive processes, functional programming should be adopted.

These kind of functions should have the following signature:

```python
def func(param, opt1, opt2, ...):
    """
    Output Tensors
    --------------
    tensor1
        Use of tensor1
    """
```

It is possible to implicitly pass input variables from `param.tensors`. However, passing tensors
by arguments is more prefered.

```python
# though this is okay
def func(param):
    return param.tensors["foo"].sum()
# the following is more preferred if param is supplied as an API requirement:
def func_more_preferred(_param, foo):
    # `_` before variable in signature tells PyCharm that variable could be unused
    return foo.sum()
```

OOP (object-oriented programs) wraps computing extensive functions. For example (though probably not suitable):

```python
def proc_dm(self, mo_coeff=None):  # defined as class member function
    if mo_coeff is None: mo_coeff = self.param.tensors["mo_coeff"]
    dm = comp_dm(self.param, mo_coeff)
    self.param.tensors.create("dm", data=dm)
    
def comp_dm(param, mo_coeff):  # computing extensive function
    nocc = param.flags["nocc"]
    return 2 * mo_coeff[:, :nocc] @ mo_coeff[:, :nocc].conj().T
```

## Temporarily Change Flags

For several cases, flags need to be changed temporarily in program. To achieve this end, one may call member function `Params.temporary_flags`. For example,

In [2]:
params = Params({"user_flag": True, "incore_A": False}, HybridDict(), {})
# change "incore_A" and add "do_cphf" temporarily
with params.temporary_flags({"incore_A": True, "do_cphf": False}):
    print(params.flags)
print(params.flags)

{'user_flag': True, 'incore_A': True, 'do_cphf': False}
{'user_flag': True, 'incore_A': False}
